In [ ]:
# Import Libraries

import pandas as pd
import numpy as np
import time
import json
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from kafka import KafkaProducer

In [ ]:
# Configuration

KAFKA_HOSTS = 'localhost:9092'
KAFKA_VERSION = (0, 10, 2)
TOPIC = "profile"
dataDirectory = "shared_data/bigdata20/followers_info.json/*.json"

In [ ]:
# Spark

spark = SparkSession.builder.master("local[*]") \
    .appName("Profile Stream Producer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1") \
    .getOrCreate()

In [ ]:
# Load Data

df = spark.read.json("shared_data/bigdata20/followers_info.json/*.json")

In [ ]:
# Write DF

kdf = df.select(F.col("id").alias("key"), F.col("sex").alias("value"))
ds = kdf \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", KAFKA_HOSTS) \
  .option("topic", TOPIC) \
  .save()

In [ ]:
kdf.count()

In [ ]:
spark.stop()